In [1]:
import pandas as pd

from statistics import mean
import statistics
import numpy as np
import os
import subprocess
import re

In [2]:

import json
with open("ast_results.json", "r") as fd:
    results = json.load(fd)

In [3]:
results[0]

{'opt': 0,
 'filename': 'PL_2mm',
 'results': {'kernel_2mm': {'w2c2': {'nqc': 0.015300023906287354,
    'ted': 0.031051517290049402},
   'wasm2c': {'nqc': 0.01562881562881563, 'ted': 0.017295219793418193}}}}

In [17]:
# First seperate OPT level and make sure all have the same entries

opt_results = {}
for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        for tool in ["w2c2", "wasm2c"]:
            if (opt, tool) not in opt_results:
                opt_results[(opt, tool)] = []
            opt_results[(opt, tool)].append(f"{filename}:{func}")
    

(0, 'w2c2') 147
(0, 'wasm2c') 147
(1, 'w2c2') 147
(1, 'wasm2c') 147
(2, 'w2c2') 147
(2, 'wasm2c') 147


In [47]:
func_results = {}


for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        func_name = f'{filename}:{func}:O{opt}'
        if func_name not in func_results:
            func_results[func_name] = {}
        for tool in ["w2c2", "wasm2c"]:
            func_results[func_name][tool] = { "opt": int(opt),
                             "nqc": result[tool]["nqc"], 
                             "ted": result[tool]["ted"]}
            #if result[tool]["ted"]< 0 and opt != 2:
            #    print(opt, tool, func_name)

func_names = []
rows = []
for func_name, columns in func_results.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")
print(df)

                          w2c2                     wasm2c                    
                           opt       nqc       ted    opt       nqc       ted
func                                                                         
PL_2mm:kernel_2mm:O0         0  0.015300  0.031052      0  0.015629  0.017295
PL_3mm:kernel_3mm:O0         0  0.015374  0.031230      0  0.015731  0.016969
adi:kernel_adi:O0            0  0.018179 -1.000000      0  0.018574 -1.000000
adpcm:abs:O0                 0  0.050481  0.077803      0  0.051095  0.078704
adpcm:encode:O0              0  0.060624 -1.000000      0  0.061121 -1.000000
...                        ...       ...       ...    ...       ...       ...
symm:kernel_symm:O2          2  0.065385  0.086228      2  0.066406  0.092345
syr2k:kernel_syr2k:O2        2  0.040293  0.057844      2  0.039778  0.044248
syrk:kernel_syrk:O2          2  0.026412  0.037116      2  0.025167  0.031677
trisolv:kernel_trisolv:O2    2  0.027100  0.044796      2  0.027

In [52]:
for opt in [0,1,2]:
    for tool in ["w2c2", "wasm2c"]:
        cnt = df[tool][(df[tool]["opt"] == opt) & (df[tool]["ted"] < 0)]["ted"].count()
        print(opt, tool, cnt)

0 w2c2 17
0 wasm2c 29
1 w2c2 14
1 wasm2c 16
2 w2c2 15
2 wasm2c 17


In [58]:
from tabulate import tabulate

table = [
    ["Tool", "Opt", "Total func", "NQC (avg)", "NQC (stdev)", "TED (avg)", "TED (stdev)", "TED (TO)"]
]
for tool in ["w2c2", "wasm2c"]:
    for opt in [0,1,2]:
        opt_tool_df = df[tool][(df[tool]["opt"] == opt)]
        cnt = opt_tool_df["opt"].count()
        avg_nqc = opt_tool_df[(opt_tool_df["nqc"] > 0)]["nqc"].mean()
        std_nqc = opt_tool_df[(opt_tool_df["nqc"] > 0)]["nqc"].std()
        avg_ted = opt_tool_df[(opt_tool_df["ted"] > 0)]["ted"].mean()
        std_ted = opt_tool_df[(opt_tool_df["ted"] > 0)]["ted"].std()
        cnt_ted = opt_tool_df[(opt_tool_df["ted"] < 0)]["ted"].count()
        table.append(
            [tool, opt, cnt, avg_nqc,  std_nqc, avg_ted, std_ted, cnt_ted]
        )
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))        

╒════════╤═══════╤══════════════╤═════════════╤═══════════════╤═════════════╤═══════════════╤════════════╕
│ Tool   │   Opt │   Total func │   NQC (avg) │   NQC (stdev) │   TED (avg) │   TED (stdev) │   TED (TO) │
╞════════╪═══════╪══════════════╪═════════════╪═══════════════╪═════════════╪═══════════════╪════════════╡
│ w2c2   │     0 │          147 │   0.16846   │     0.205529  │   0.182024  │     0.172026  │         17 │
├────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ w2c2   │     1 │          147 │   0.229202  │     0.194485  │   0.231683  │     0.157311  │         14 │
├────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ w2c2   │     2 │          147 │   0.22258   │     0.19972   │   0.222599  │     0.167707  │         15 │
├────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ wasm2c │     0 │          147 │   0